In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from swarm_sim import *

%matplotlib inline 

Load the data and format it as Swarm object

In [ ]:
PATH = '..\data\Traces_Nanosatellites\\track_'
satellites = {}

with tqdm(total=100, desc='Extracting data') as pbar:
    for i in range(0,100):
        df = pd.read_csv(PATH+str(i)+'.csv', sep=',', header=0)
        df['coords'] = ['x','y','z']
        satellites[i] = df.set_index('coords', drop=True)
        pbar.update(1)
    
DURATION = satellites[0].columns.tolist()
CHUNKS = 2000     # Number of timestamps to analyse
NB_NODES = 100

satellites[0].head()

In [ ]:
CONNECTION_RANGE = 30000
swarm_data = {}

with tqdm(total = len(DURATION), desc = 'Converting to Swarm') as pbar:
    for t in DURATION:
        swarm_data[int(t)] = Swarm(
            connection_range=CONNECTION_RANGE, 
            nodes=[Node(id, node[t].x, node[t].y, node[t].z) for id,node in satellites.items()]
            )
        pbar.update(1)

print(swarm_data[0])

In [ ]:
swarm_chunk = {}
for t in range(CHUNKS):
    swarm_chunk[t] = swarm_data[t]

Compute node degree distribution & evolution over the whole duration (optional)

In [ ]:

fig,ax = plt.subplots(figsize=(6,4))

bp = ax.boxplot(degree_range.values(), 
            vert=False, 
            widths=0.5,
            meanline=True, 
            showmeans=True, 
            meanprops={'color':'red','ls':'--'}
            )
ax.legend([bp['medians'][0], bp['means'][0]], ['Median', 'Average'])
ax.set_yticklabels([20,40,60]) #degree_range.keys()
ax.set_ylabel('Connection range (km)')
ax.set_xlabel('Node degree')

#fig.suptitle('Node degree distribution')

In [ ]:
with tqdm(total=len(DURATION), desc='Neighbor Discovery') as pbar:
    for t,swarm in swarm_data.items():
        neighbor_matrix = swarm.neighbor_matrix()
        pbar.update(1)

In [ ]:
degree_distrib = {}

with tqdm(total=len(DURATION), desc='Degree distribution') as pbar:
    for t,swarm in swarm_data.items():
        degree_distrib[t] = swarm.degree()
        pbar.update(1)

In [ ]:
dmin = [np.min(d) for d in degree_distrib.values()]
dq1 = [np.quantile(d, 0.25) for d in degree_distrib.values()]
dmed = [np.median(d) for d in degree_distrib.values()]
dq3 = [np.quantile(d, 0.75) for d in degree_distrib.values()]
dmax = [np.max(d) for d in degree_distrib.values()]

In [ ]:
idx = np.arange(0,100000,10)

plt.figure(figsize=(6,6))
plt.plot(idx, dmin, label='min', c='blue', lw=1, ls=':')
plt.plot(idx, dq1, label='Q1', c='blue', lw=1)
plt.plot(idx, dmed, label='median', c='green', lw=2)
plt.plot(idx, dq3, label='Q3', c='red', lw=1)
plt.plot(idx, dmax, label='max', c='red', lw=1, ls=':')
plt.legend(loc='upper right')
plt.xlabel('Time (s)')
plt.ylabel('Degree')

Neighbor discovery on smaller chunk of data

In [ ]:
def compute_neighbor_matrix(swarm_chunk, connection_range=None):
    neighbor_matrix = {}
    with tqdm(total=CHUNKS, desc='Computing Neighbor matrix') as pbar:
        for t,swarm in swarm_chunk.items():
            neighbor_matrix[t] = swarm.neighbor_matrix(connection_range)
            pbar.update(1)
    return neighbor_matrix

def compute_swarm_degree(swarm_chunk):
    swarm_degree = {}
    with tqdm(total=CHUNKS, desc='Computing Swarm degree') as pbar:    
        for t,swarm in swarm_chunk.items():
            swarm_degree[t] = swarm.degree()
            pbar.update(1)
    return swarm_degree

def init_network(swarm_chunk, cr=None):
    if not cr:
        cr = swarm_chunk[0].connection_range
    print('\nConnection range:', cr)
    neighbor_matrix = compute_neighbor_matrix(swarm_chunk, connection_range=cr)
    swarm_degree = compute_swarm_degree(swarm_chunk)
    sum_data = [e for t in swarm_degree.keys() for e in swarm_degree[t]]
    print('Minimum number of neighbors:', np.min(sum_data))
    print('Maximum number of neighbors:', np.max(sum_data))
    print('Average number of neighbors:', np.mean(sum_data))
    node_ict = []
    for i in range(NB_NODES):
        node_data = [swarm_degree[t][i] for t in swarm_degree.keys()]
        node_ict.append(node_data.count(0) / float(CHUNKS)*100)
    mean_ict = np.mean(node_ict)
    print('Mean inter-contact time:', mean_ict)

def compute_swarm_kvicinity(swarm_chunk, depth=1):
    swarm_kv = {}
    with tqdm(total=CHUNKS, desc=f'Computing Swarm k-vicinity (depth {depth})') as pbar:    
        for t,swarm in swarm_chunk.items():
            swarm_kv[t] = swarm.k_vicinity(depth)
            pbar.update(1)
    return swarm_kv

def avg_kvicinity(swarm_chunk, depth=1):
    swarm_kv = compute_swarm_kvicinity(swarm_chunk, depth)
    avg_kv = {}
    for i in range(NB_NODES):
        avg_kv[i] = np.mean([swarm_kv[t][i] for t in swarm_kv.keys()])
    return avg_kv

In [ ]:
init_network(swarm_chunk, cr=40000)

In [ ]:
k_vicinities= {}
for i in range(1,4):
    k_vicinities[i] = avg_kvicinity(swarm_chunk, depth=i)

In [ ]:
plt.figure(figsize=(6,6))
idx = list(k_vicinities[1].values()) # Degree
colors = {1:'blue',2:'green',3:'red'}
for k,v in k_vicinities.items():
    plt.scatter(idx, list(v.values()), c=colors[k], lw=1, label='k='+str(k))
plt.legend(loc='upper left')
plt.xlabel('Node degree')
plt.ylabel('K-vicinity')
#plt.title('Evolution of k-vicinity in function of node degree')

In [ ]:
# Thorough analysis, optional

ranges = [60000,50000,40000,30000,20000,15000,10000]
swarm_ict = {}
swarm_neigh = {}
for cr in ranges:
    print('\nConnection range:', cr)
    neighbor_matrix = compute_neighbor_matrix(swarm_chunk, connection_range=cr)
    swarm_degree = compute_swarm_degree(swarm_chunk)
    sum_data = [e for t in swarm_degree.keys() for e in swarm_degree[t]]
    print('Minimum number of neighbors:', np.min(sum_data))
    print('Maximum number of neighbors:', np.max(sum_data))
    print('Average number of neighbors:', np.mean(sum_data))
    swarm_neigh[cr] = (np.min(sum_data), np.max(sum_data), np.mean(sum_data))
    node_ict = []
    for i in range(NB_NODES):
        node_data = [swarm_degree[t][i] for t in swarm_degree.keys()]
        node_ict.append(node_data.count(0) / float(CHUNKS)*100)
    mean_ict = np.mean(node_ict)
    print('Mean inter-contact time:', mean_ict)
    swarm_ict[cr] = mean_ict

Network characterization
1. Intercontact time and disponibility

In [ ]:
def pairwise_dispo(neighbor_matrix):
    dispos = {} # (n1,n2):avg_dispo (%)
    for n1 in range(NB_NODES):
        pairs = [set(pair) for pair in dispos.keys()]
        for n2 in range(NB_NODES):
            if n1 != n2 and set([n1,n2]) not in pairs:
                pair_dispo = [neighbor_matrix[t][n1][n2] for t in neighbor_matrix.keys()]
                dispos[(n1,n2)] = np.mean(pair_dispo)*100
    return dispos    

def pairwise_ict(neighbor_matrix):
    ict = {} # (n1,n2):avg_ict (%)
    for n1 in range(NB_NODES):
        pairs = [set(pair) for pair in ict.keys()]
        for n2 in range(NB_NODES):
            if n1 != n2 and set([n1,n2]) not in pairs:
                pair_ict = [neighbor_matrix[t][n1][n2] for t in neighbor_matrix.keys()]
                ict[(n1,n2)] = (1-np.mean(pair_ict))*100
    return ict    
            

In [ ]:
matrix = compute_neighbor_matrix(swarm_chunk, connection_range=CONNECTION_RANGE)


In [ ]:
dispos = pairwise_dispo(matrix)
pair_dispo = sorted(dispos.values(), reverse=True)
cr = CONNECTION_RANGE/1000

plt.figure(figsize=(8,8))
plt.plot(pair_dispo)
plt.xlabel('Pair ID')
plt.ylabel('Disponibility (%)')
plt.title(f'Average disponibility for each pair of nodes (range={cr} km)')

In [ ]:
ict = pairwise_ict(matrix)


In [ ]:
cr = CONNECTION_RANGE/1000
count, bins_count = np.histogram(list(ict.values()), bins=50)
pdf = count / sum(count)
cdf = np.cumsum(pdf)

fig, ax1 = plt.subplots(figsize=(6,5))

ax1.hist(ict.values(), bins=50)
ax1.set_ylabel('Number of pairs', color='blue')
ax1.set_xlabel('ICT (%)')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
ax2.plot(bins_count[1:], cdf, color='red', lw=2)
ax2.set_ylabel('CDF', color='red')
ax2.tick_params(axis='y', labelcolor='red')

#fig.suptitle(f'Average ICT distribution for each pair of nodes (range={cr} km)')
fig.tight_layout()

2. Peer-to-peer distances


In [ ]:
def compute_distance_matrix(swarm_chunk):
    distance_matrix = {}
    with tqdm(total=CHUNKS, desc='Computing Distance matrix') as pbar:
        for t,swarm in swarm_chunk.items():
            distance_matrix[t] = swarm.distance_matrix()
            pbar.update(1)
    return distance_matrix

In [ ]:
dm = compute_distance_matrix(swarm_chunk)

In [ ]:
distance_data = [e for data in dm.values() for node in data for e in node]

In [ ]:
plt.figure(figsize=(8,8))
plt.hist(distance_data, bins=50)
plt.xlabel('Distance (m)')
plt.ylabel('Occurrences')
plt.title(f'Peer-to-peer distance distribution')

In [ ]:
fig,ax = plt.subplots(figsize=(8,4))

bp = ax.boxplot(distance_data, 
            vert=False, 
            meanline=True, 
            showmeans=True, 
            meanprops={'color':'red','ls':'--'}
            )
ax.legend([bp['medians'][0], bp['means'][0]], ['Median', 'Average'])
ax.set_xlabel('Distance (m)')

fig.suptitle('Peer-to-peer distance distribution')

Clustering coefficient distribution

In [ ]:
# http://mlg.eng.cam.ac.uk/pub/pdf/HueBorKriGha08.pdf
"""
Graph mining for finding representative subgraphs
Interesting metrics for Metropolis algorithm for sampling and evaluating:
    x graph degree distribution d_avg, d(v)
    x clustering coefficient Cv of node v with degree d(v): number of edges actually existing between neighbors of v
      divided by the maximum possible number of such edges between its neighbors d(v)*(d(v)-1)/2
    x clustering coefficient distribution Cd = avg(Cv)
    - graphlet distribution: given a graph G, a k-graphlet is a connected and induced subgraph of G of size k.
      Use the distribution of 3,4 and 5-graphlets in the graph.
For evaluation purpose only:
    - diameter of the graph: maximum shortest path length between any pair of nodes in the graph
"""

In [ ]:
t = '0'
swarm_20 = Swarm(
    connection_range=20000, 
    nodes=[Node(id, node[t].x, node[t].y, node[t].z) for id,node in satellites.items()]
    )
print(swarm_20)

nm20 = swarm_20.neighbor_matrix()
d20 = swarm_20.degree()
print('Average node degree:', np.mean(d20))

cc20 = swarm_20.cluster_coef()
print('Average clustering coefficient:', np.mean(cc20))

In [ ]:
t = '0'
swarm_30 = Swarm(
    connection_range=30000, 
    nodes=[Node(id, node[t].x, node[t].y, node[t].z) for id,node in satellites.items()]
    )
print(swarm_30)

nm30 = swarm_30.neighbor_matrix()
d30 = swarm_30.degree()
print('Average node degree:', np.mean(d30))

cc30 = swarm_30.cluster_coef()
print('Average clustering coefficient:', np.mean(cc30))

In [ ]:
t = '0'
swarm_40 = Swarm(
    connection_range=40000, 
    nodes=[Node(id, node[t].x, node[t].y, node[t].z) for id,node in satellites.items()]
    )
print(swarm_40)

nm40 = swarm_40.neighbor_matrix()
d40 = swarm_40.degree()
print('Average node degree:', np.mean(d40))

cc40 = swarm_40.cluster_coef()
print('Average clustering coefficient:', np.mean(cc40))

In [ ]:
t = '0'
swarm_50 = Swarm(
    connection_range=50000, 
    nodes=[Node(id, node[t].x, node[t].y, node[t].z) for id,node in satellites.items()]
    )
print(swarm_50)

nm50 = swarm_50.neighbor_matrix()
d50 = swarm_50.degree()
print('Average node degree:', np.mean(d50))

cc50 = swarm_50.cluster_coef()
print('Average clustering coefficient:', np.mean(cc50))

In [ ]:
t = '0'
swarm_60 = Swarm(
    connection_range=60000, 
    nodes=[Node(id, node[t].x, node[t].y, node[t].z) for id,node in satellites.items()]
    )
print(swarm_60)

nm60 = swarm_60.neighbor_matrix()
d60 = swarm_60.degree()
print('Average node degree:', np.mean(d60))

cc60 = swarm_60.cluster_coef()
print('Average clustering coefficient:', np.mean(cc60))

In [ ]:
fig,ax = plt.subplots(nrows=2, ncols=5, figsize=(17,7))
fig.suptitle('Clustering coefficient distribution for various connection ranges')

ax[0][0].set_title('20 km')
ax[0][0].hist(cc20, bins=30, density=False)
bp = ax[1][0].boxplot(cc20, 
            vert=False, 
            meanline=True, 
            showmeans=True, 
            meanprops={'color':'red','ls':'--'}
            )
ax[1][0].legend([bp['medians'][0], bp['means'][0]], ['Median', 'Average'])

ax[0][1].set_title('30 km')
ax[0][1].hist(cc30, bins=30, density=False)
bp = ax[1][1].boxplot(cc30, 
            vert=False, 
            meanline=True, 
            showmeans=True, 
            meanprops={'color':'red','ls':'--'}
            )
ax[1][1].legend([bp['medians'][0], bp['means'][0]], ['Median', 'Average'])

ax[0][2].set_title('40 km')
ax[0][2].hist(cc40, bins=30, density=False)
bp = ax[1][2].boxplot(cc40, 
            vert=False, 
            meanline=True, 
            showmeans=True, 
            meanprops={'color':'red','ls':'--'}
            )
ax[1][2].legend([bp['medians'][0], bp['means'][0]], ['Median', 'Average'])

ax[0][3].set_title('50 km')
ax[0][3].hist(cc50, bins=30, density=False)
bp = ax[1][3].boxplot(cc50, 
            vert=False, 
            meanline=True, 
            showmeans=True, 
            meanprops={'color':'red','ls':'--'}
            )
ax[1][3].legend([bp['medians'][0], bp['means'][0]], ['Median', 'Average'])

ax[0][4].set_title('60 km')
ax[0][4].hist(cc60, bins=30, density=False)
bp = ax[1][4].boxplot(cc60, 
            vert=False, 
            meanline=True, 
            showmeans=True, 
            meanprops={'color':'red','ls':'--'}
            )
ax[1][4].legend([bp['medians'][0], bp['means'][0]], ['Median', 'Average'])

In [ ]:
cc = swarm_20.connected_components()
print(cc)

In [ ]:
swarm_20.plot_edges()


In [ ]:
cc = swarm_30.connected_components()
print(cc)

In [ ]:
swarm_30.plot_edges()


In [ ]:
cc = swarm_40.connected_components()
print(cc)

In [ ]:
swarm_40.plot_edges()